In [8]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

In [9]:
# load Piazza
driver = webdriver.Chrome()
driver.get('https://piazza.com/class/m5l90em6d0u656') # enter URL of desired class

In [10]:
# LOGIN
driver.find_element(By.LINK_TEXT, "Login with your school's credentials (SSO)").click()
time.sleep(2)
driver.find_element(By.LINK_TEXT, "Georgia Institute of Technology").click()
time.sleep(2)

In [11]:
# INPUT CREDENTIALS
username_input = driver.find_element(By.ID, "username")
username_input.clear()
username_input.send_keys("eliu309") # change to gatech username

password_input = driver.find_element(By.ID, "password")
password_input.clear()
password_input.send_keys("LWenXin444#") # change to gatech password

time.sleep(2)
driver.find_element(By.NAME, "submitbutton").click()

In [12]:
# after 2FA via Duo Mobile, click "Yes, this is my device"
driver.find_element(By.ID, "trust-browser-button").click()

In [21]:
# filter by folder
driver.find_element(By.ID, "topbar_folder_4").click()

In [22]:
# expand all sections in LHS
from selenium.webdriver.common.action_chains import ActionChains
post_groups = driver.find_elements(By.XPATH, "//div[@data-id='post_group']")

for post_group in post_groups:
    try:
        ActionChains(driver).move_to_element(post_group).click().perform()
        time.sleep(1)
    except Exception as e:
        print(f"Error clicking element: {e}")

print("All post groups clicked!")

All post groups clicked!


In [23]:
# retrieve all posts
post_items = driver.find_elements(By.XPATH, "//li[@data-id='post_group_item']")
print(len(post_items))

67


In [26]:
# scrape all posts
from bs4 import BeautifulSoup

for i in range(75, len(post_items)):
    print(i)
    post_items[i].click()
    # question
    try:
        qa = driver.find_element(By.XPATH, "//div[@class='col questions-and-answers']").find_element(By.XPATH, ".//div[@class='content container-fluid']")
        question_content = qa.get_attribute("innerHTML")
        question_soup = BeautifulSoup(question_content, "html.parser")
        clean_question = question_soup.get_text(strip=True).replace("Actions", "")
        print(clean_question)
    except:
        print("err question")
        clean_question = ""

    # student answer
    try: 
        s_answer = driver.find_element(By.XPATH, "//*[@data-id='s_answer']").find_element(By.XPATH, ".//div[@class='content container-fluid']")
        s_answer_content = s_answer.get_attribute("innerHTML")
        s_answer_soup = BeautifulSoup(s_answer_content, "html.parser")
        clean_s_answer = s_answer_soup.get_text(strip=True).replace("Actions", "")
        print(clean_s_answer)
    except:
        print("err s_answer")
        clean_s_answer = ""

    # instructor answer
    try: 
        i_answer = driver.find_element(By.XPATH, "//*[@data-id='i_answer']").find_element(By.XPATH, ".//div[@class='content container-fluid']")
        i_answer_content = i_answer.get_attribute("innerHTML")
        i_answer_soup = BeautifulSoup(i_answer_content, "html.parser")
        clean_i_answer = i_answer_soup.get_text(strip=True)

        print(clean_i_answer)
    except:
        print("err i_answer")
        clean_i_answer = ""

    # follow up discussions
    try: 
        follow_up = driver.find_element(By.XPATH, "//*[@aria-label='Followup Discussions']")
        follow_up_content = follow_up.get_attribute("innerHTML")
        follow_up_soup = BeautifulSoup(follow_up_content, "html.parser")
        clean_follow_up = follow_up_soup.get_text(strip=True)
        clean_follow_up = clean_follow_up.replace("followup discussions", "")
        clean_follow_up = clean_follow_up.replace("for lingering questions and comments", "")
        clean_follow_up = clean_follow_up.replace("Start a new followup discussion", "")
        clean_follow_up = clean_follow_up.replace("Resolved", "")
        clean_follow_up = clean_follow_up.replace("Unresolved", "")
        clean_follow_up = clean_follow_up.replace("helpful!", "")

        print(clean_follow_up)
    except:
        clean_follow_up = ""

    file_content = f"""Question: 
    {clean_question}

    Student Answer: 
    {clean_s_answer}

    Instructor Answer: 
    {clean_i_answer}

    Followup Discussions:
    {clean_follow_up}
    """

    # store as a text file
    file_name = f"./lab4/post_{i + 1}.txt"
    with open(file_name, "w", encoding="utf-8") as file:
        file.write(file_content)

    time.sleep(0.2)
